# Confidential ML Training Demo (Data Owner 1)

This notebook is the Data Owners part of the *Confidential ML Training Demo* showing how a simple logistic regression classifier can be trained while keeping the training data provably confidential. The demo requires the [Training Client API](https://github.com/decentriq/avato-python-client-training) and its dependencies to be installed.  

## 1 - Import dependencies and submission code

In [ ]:
import os
import pandas as pd
from avato import Client
from avato import Secret
from avato_training import Training_Instance
import example

dataowner1_username = os.getenv('DATAOWNER1_ID')
dataowner1_password = os.getenv('DATAOWNER1_PASSWORD')

# This is the hash of the code
expected_measurement = "8e6c77bd904826526918a7e8fd4ccdc35cb8f8b5e0241fc78749e80dedf00cdf"

# The datafile
dataowner1_file = "test-data/wine-dataowner1.csv"

backend_host = "avato.uksouth.cloudapp.azure.com" 
backend_port = 15005 

## 2 - Set instance id received from Analyst

In [ ]:
# TO MODIFY in accordance to Instance ID created and provided 
#   by the analyst (Analyst notebook, Section 2, Instance ID: [...])
instance_id_from_analyst = "9a1deb5e51b73ec56977230c19827b960280857caaf794ea1cda03ff91063658"

## 3 - Submit Data

#### Create client and connect to the instance created by the analyst

In [ ]:
# Create client
dataowner1_client = Client(
    username=dataowner1_username,
    password=dataowner1_password,
    instance_types=[Training_Instance],
    backend_host=backend_host,
    backend_port=backend_port
)

# Connect to instance (using ID from the analyst user)
dataowner1_instance = dataowner1_client.get_instance(instance_id_from_analyst)

#### Verify security and create own keypair

In [ ]:
# Verify security.
dataowner1_instance.validate_fatquote(
    expected_measurement=expected_measurement,
    accept_debug=True,
    accept_group_out_of_date=True
)

# Create and set public-private keypair for secure communication.
dataowner1_secret = Secret()
dataowner1_instance.set_secret(dataowner1_secret)

#### Get data format from the enclave

In [ ]:
# Get data format from the enclave
data_format = dataowner1_instance.get_data_format()
print("Data format:\n{}".format(data_format))

#### Load data

In [ ]:
# Load data
df = pd.read_csv(dataowner1_file)
df.head(2)

#### Submit data

In [ ]:
(ingested_rows, failed_rows) = dataowner1_instance.submit_data(df)
print("\nNumber of successfully ingested rows: {}, number of failed rows: {}".format(ingested_rows, failed_rows))